# 第6章 所得分布はどのように生じるか(3)：シミュレーションモデル

第4章（乗算モデル → 対数正規分布）と第5章（交換モデル → 指数分布）で学んだ  
所得分布の生成メカニズムを、Rでシミュレーションして確かめます。

**この章の目標**:
- `sample()`, `replicate()` を使ったシミュレーションの基本を身につける
- 乗算過程が対数正規分布を生成することをシミュレーションで確認する
- 交換過程が指数分布を生成することをシミュレーションで確認する
- 2つのモデルの結果を比較する

*このノートブックは Claude Opus 4.6 を利用して作成しました。*

## 6.1 シミュレーションの準備

まず、シミュレーションに必要な基本操作を確認します。

In [ ]:
library(tidyverse)

### `set.seed()`: 乱数の再現性

シミュレーションでは乱数を使いますが、`set.seed()` で**乱数の種**を固定すると、毎回同じ結果が得られます。

In [ ]:
set.seed(2026)
rnorm(5)

In [ ]:
# 同じシードを設定すると同じ結果が得られる
set.seed(2026)
rnorm(5)

### `sample()`: 離散的なランダム抽出

In [ ]:
# コイン投げを10回
set.seed(2026)
sample(c("表", "裏"), size = 10, replace = TRUE)

In [ ]:
# サイコロを1000回振って度数を数える
set.seed(2026)
dice <- sample(1:6, size = 1000, replace = TRUE)
table(dice)

In [ ]:
barplot(table(dice), col = "skyblue",
        main = "サイコロ1000回の結果")

### `replicate()`: 同じ操作の反復

シミュレーションの中核となる関数です。「同じ確率的な操作を何度も繰り返し、その結果の分布を調べる」ために使います。

In [ ]:
# 「10個の正規乱数の平均」を1000回繰り返す
set.seed(2026)
sample_means <- replicate(1000, mean(rnorm(10)))

In [ ]:
hist(sample_means, breaks = 30, prob = TRUE,
     col = "lightgreen",
     main = "標本平均の分布（n=10, 1000回）",
     xlab = "標本平均")
curve(dnorm(x, 0, 1/sqrt(10)), col = "red", lwd = 2, add = TRUE)

---

## 6.2 乗算モデルのシミュレーション

第4章で学んだ乗算モデルをシミュレーションします。

**モデルの設定**（講義ノート §4.6）:
- 確率 $p$ で手持ちを $b$ 倍できる試行を $n$ 回繰り返す
- 個人 $i$ の $n$ 回後の総所得: $Y_i = b^{W_i}$（$W_i$ は勝った回数）
- $W_i \sim \mathrm{Binomial}(n, p)$ なので、$Y_i$ は近似的に**対数正規分布**に従う

### 1人分の乗算ゲーム

In [ ]:
# パラメータ設定
b <- 1.3    # 勝ったときの倍率
n <- 30     # 試行回数
p <- 0.7    # 勝つ確率

In [ ]:
# 1人分のゲームをシミュレーション
set.seed(2026)
wins <- sample(c(0, 1), size = n, replace = TRUE, prob = c(1 - p, p))

In [ ]:
# 勝った回数
W <- sum(wins)
W

In [ ]:
# 総所得
Y <- b ^ W
Y

### 多数の人に対するシミュレーション

In [ ]:
# N人分の乗算ゲームをシミュレーション
N <- 10000   # 人数

set.seed(2026)
# 各人の勝ち数は二項分布に従う
W_all <- rbinom(N, size = n, prob = p)

In [ ]:
# 各人の総所得
Y_all <- b ^ W_all

In [ ]:
# 所得分布のヒストグラム
hist(Y_all, breaks = 50, prob = TRUE,
     col = "salmon",
     main = "乗算モデルによる所得分布",
     xlab = "所得 Y")

右に裾の長い分布が見えます。次に対数をとってみましょう。

In [ ]:
# 対数所得の分布
hist(log(Y_all), breaks = 50, prob = TRUE,
     col = "skyblue",
     main = "対数所得 log(Y) の分布",
     xlab = "log(Y)")

# 理論的な正規分布を重ねる
mu    <- n * p * log(b)
sigma <- sqrt(n * p * (1 - p)) * log(b)
curve(dnorm(x, mu, sigma), col = "red", lwd = 2, add = TRUE)

> **確認**: $\log Y$ の分布は、理論値 $\mathrm{Normal}(np\log b,\; \sqrt{np(1-p)}\log b)$ の正規分布（赤い曲線）とよく一致しています。  
> これが「乗算過程 → 対数正規分布」の中心極限定理による説明です。

### 基本統計量の確認

In [ ]:
mean(Y_all)

In [ ]:
median(Y_all)

In [ ]:
# 平均 > 中央値 → 右に裾の長い分布
mean(Y_all) > median(Y_all)

### パラメータを変えてみる

倍率 $b$ を変えると、所得分布の形状がどう変わるかを確認します。  
（講義ノート: $\partial G / \partial b > 0$、倍率が大きいほど不平等度が増す）

In [ ]:
set.seed(2026)
par(mfrow = c(1, 3))

for (b_val in c(1.1, 1.3, 2.0)) {
  W_sim <- rbinom(N, size = n, prob = p)
  Y_sim <- b_val ^ W_sim
  hist(Y_sim, breaks = 40, prob = TRUE,
       col = "salmon", main = paste("b =", b_val),
       xlab = "Y")
}

par(mfrow = c(1, 1))

---

## 6.3 交換モデルのシミュレーション

第5章で学んだ Dragulescu & Yakovenko (2000) の交換モデルをシミュレーションします。

**モデルの設定**（講義ノート 第5章）:
- $N$ 人のエージェントが初期の所持金を持つ
- 毎ラウンド、ランダムにペアを組み、一方から他方へ金額 $\Delta m$ を移転する
- 貨幣の総量は保存される（ゼロサム）
- 負債は許容されない（$m_i \geq 0$）
- 十分な回数を繰り返すと、**指数分布** $P(m) \propto e^{-m/T}$（$T = M/N$）に収束する

In [ ]:
# パラメータ設定
N_agent  <- 500        # エージェント数
M_total  <- 500000     # 貨幣の総量
T_mean   <- M_total / N_agent  # 1人あたり平均（= 指数分布のパラメータ T）
n_rounds <- 100000     # 交換ラウンド数

In [ ]:
# 初期状態：全員が同額を持つ
money <- rep(T_mean, N_agent)

In [ ]:
# 交換シミュレーション
set.seed(2026)

for (t in 1:n_rounds) {
  # ランダムに2人を選ぶ
  pair <- sample(N_agent, 2)
  i <- pair[1]
  j <- pair[2]
  
  # 移転額をランダムに決定（i が持っている額の範囲内）
  delta_m <- runif(1, 0, money[i])
  
  # 移転
  money[i] <- money[i] - delta_m
  money[j] <- money[j] + delta_m
}

In [ ]:
# 保存則の確認：総額は変わっていないか？
sum(money)

In [ ]:
# 結果のヒストグラム
hist(money, breaks = 40, prob = TRUE,
     col = "lightgreen",
     main = "交換モデルによる貨幣分布",
     xlab = "所持金 m")

# 理論的な指数分布を重ねる
curve(dexp(x, rate = 1 / T_mean), col = "red", lwd = 2, add = TRUE)

> **確認**: シミュレーション結果（ヒストグラム）が理論的な指数分布（赤い曲線）とよく一致しています。  
> 全員が同額からスタートしても、ランダムな交換だけで不平等な分布が生じることがわかります。

### 対数スケールでの確認

指数分布であれば、対数スケールのプロットで直線になるはずです（Dragulescu & Yakovenko, 2000, Fig.1 の挿入図に対応）。

In [ ]:
# 経験的な累積分布を対数スケールでプロット
money_sorted <- sort(money)
ecdf_vals <- 1 - (1:N_agent) / N_agent

plot(money_sorted, ecdf_vals, log = "y",
     pch = 20, cex = 0.5, col = "steelblue",
     main = "補累積分布（対数スケール）",
     xlab = "所持金 m",
     ylab = "P(M > m)")

# 理論値（指数分布の生存関数）
curve(exp(-x / T_mean), col = "red", lwd = 2, add = TRUE)

---

## 6.4 モデルの比較

乗算モデルと交換モデルの結果を並べて比較してみましょう。

In [ ]:
par(mfrow = c(1, 2))

# 乗算モデル
hist(Y_all, breaks = 50, prob = TRUE,
     col = "salmon",
     main = "乗算モデル\n（対数正規分布）",
     xlab = "所得 Y")

# 交換モデル
hist(money, breaks = 40, prob = TRUE,
     col = "lightgreen",
     main = "交換モデル\n（指数分布）",
     xlab = "所持金 m")

par(mfrow = c(1, 1))

### 基本統計量の比較

In [ ]:
# 乗算モデル
cat("【乗算モデル】\n")
cat("  平均:", mean(Y_all), "\n")
cat("  中央値:", median(Y_all), "\n")
cat("  標準偏差:", sd(Y_all), "\n")

In [ ]:
# 交換モデル
cat("【交換モデル】\n")
cat("  平均:", mean(money), "\n")
cat("  中央値:", median(money), "\n")
cat("  標準偏差:", sd(money), "\n")

### ggplot2 での比較プロット

In [ ]:
# 2つのモデルの結果を1つのデータフレームにまとめる
# スケールを揃えるため、交換モデルの結果を平均が同じになるように正規化
df_compare <- data.frame(
  value = c(Y_all / mean(Y_all), money / mean(money)),
  model = rep(c("乗算モデル", "交換モデル"), each = c(length(Y_all), length(money)))
)

In [ ]:
ggplot(df_compare, aes(x = value, fill = model)) +
  geom_histogram(bins = 50, alpha = 0.6, position = "identity") +
  labs(title = "乗算モデル vs 交換モデル（平均で正規化）",
       x = "所得 / 平均所得",
       y = "度数",
       fill = "モデル") +
  theme_classic()

## まとめ

| | 乗算モデル | 交換モデル |
|---|---|---|
| メカニズム | 各期の所得 = 前期 × ランダムな倍率 | ランダムなペア間での貨幣移転 |
| 理論分布 | 対数正規分布 | 指数分布 |
| 分布の形状 | 右に裾が長い（中間層にピーク） | 右に裾が長い（低額にピーク） |
| 背景 | 資本の累積的成長（ピケティ $r > g$） | 統計力学のアナロジー（保存則） |

いずれのモデルも、単純なルールから「少数の高所得者と多数の低〜中所得者」という  
現実の所得分布の特徴を再現できることが確認できました。